This is needed to run the codes from fairseq_signals when testing ECG-FM model.

I still cannot manage to run it yet

In [ ]:
import os
import pandas as pd
import ast
import wfdb
from scipy.io import savemat
import numpy as np
import random
import argparse
from data import ptbxl
import os
from model.huggingface.utils import download_model
from fairseq_signals import models, tasks
from fairseq_signals.utils import checkpoint_utils
import torch

In [ ]:
try:
  ptbxl.PTBXL()
except:
  os.chdir('../')

In [ ]:
dataset = ptbxl.PTBXL(sampling_rate=ptbxl.SamplingRate.HZ_500)
record = dataset.load_record(1)

# Try manifest generation

In [ ]:


data_dir = "dataset/ptb-xl"
output = "dataset/ptb-xl/processed/ptbxl_manifest.tsv"
output_path = os.path.abspath(output)
output_dir = os.path.dirname(output_path)
if output_dir and not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Load metadata CSV files
db_csv = os.path.join(data_dir, "ptbxl_database.csv")
scp_csv = os.path.join(data_dir, "scp_statements.csv")
if not os.path.isfile(db_csv) or not os.path.isfile(scp_csv):
    raise FileNotFoundError("Could not find ptbxl_database.csv or scp_statements.csv in the provided data_dir")

# Read scp_statements.csv to get diagnostic class info
scp_df = pd.read_csv(scp_csv, index_col=0)  # use SCP code as index
# Filter to diagnostic statements only
diag_df = scp_df[scp_df['diagnostic'] == 1.0]

# Read ptbxl_database.csv with scp_codes as a dictionary
ptbxl_df = pd.read_csv(db_csv, converters={'scp_codes': ast.literal_eval})

# Prepare label mapping and output container
all_subclasses = set()

# First pass: gather all diagnostic subclass labels for each record
records_info = []  # will hold tuples of (record_path, segment_length, label_indices)
for _, row in ptbxl_df.iterrows():
    codes_dict: dict[str, int] = row['scp_codes']
    # Determine diagnostic subclasses present in this record's annotations
    label_subclasses = set()
    for code, likelihood in codes_dict.items():
        if likelihood > 0 and code in diag_df.index:
            subclass = diag_df.loc[code, 'diagnostic_subclass']
            if isinstance(subclass, str) and subclass != '':
                label_subclasses.add(subclass)
    if not label_subclasses:
        # skip records with no diagnostic labels (e.g. only rhythm/form statements)
        continue
    all_subclasses.update(label_subclasses)
    # Store record info for processing (filename_hr and subclasses set)
    records_info.append((row['filename_hr'], label_subclasses))

# Create a stable mapping from subclass label to numeric ID
all_subclasses = sorted(all_subclasses)  # sort for consistent ordering
subclass_to_id = {subclass: idx for idx, subclass in enumerate(all_subclasses)}

# Set random seed for reproducible cropping
random.seed(0)
# Process each record: read waveform, crop 5s segment, save .mat, prepare manifest line
with open(output_path, 'w') as manifest_file:
    for file_path, label_subclasses in records_info:
        # Construct full path to the WFDB record (500 Hz data)
        record_base = os.path.join(data_dir, file_path) 
        # Read the ECG record using WFDB (this finds the .dat and .hea files)
        try:
            record = wfdb.rdrecord(record_base)
        except Exception as e:
            print(f"Warning: skipping record {file_path} due to read error: {e}")
            continue
        signal = record.p_signal  # numpy array of shape (n_samples, n_leads)
        if signal is None:
            # If p_signal is None, try reading as integer signals
            signal, _ = wfdb.rdsamp(record_base)
        if signal is None:
            print(f"Warning: no signal data for record {file_path}, skipping.")
            continue

        # Determine segment length for 5 seconds (assuming sampling rate 500 Hz)
        fs = int(record.fs) if hasattr(record, 'fs') else 500  # fallback to 500 if not available
        segment_samples = 5 * fs
        total_samples = signal.shape[0]
        if total_samples < segment_samples:
            # If the record is shorter than 5s, skip (not expected for PTB-XL)
            continue
        # Randomly choose a 5s segment from the 10s recording
        start_idx = 0
        if total_samples > segment_samples:
            start_idx = random.randint(0, total_samples - segment_samples)
        segment = signal[start_idx : start_idx + segment_samples]

        # Save the segment to a .mat file in the output directory
        # Use the record base name (without _hr suffix) as the file name
        base_name = os.path.basename(file_path)
        if base_name.endswith('_hr'):
            base_name = base_name[:-3]  # drop the '_hr'
        elif base_name.endswith('_lr'):
            base_name = base_name[:-3]  # drop '_lr' if somehow using 100Hz data
        mat_filename = base_name + ".mat"
        mat_path = os.path.join(output_dir, mat_filename)
        # Save using SciPy savemat
        savemat(mat_path, {"ecg": segment})

        # Prepare label string: one or multiple label indices
        label_ids = [subclass_to_id[sub] for sub in sorted(label_subclasses)]
        # Join multiple labels by comma (if only one, it will just be that number)
        label_str = ",".join(str(x) for x in label_ids)
        # Write the manifest line: "<file>\t<length>\t<labels>"
        # Use absolute path for the file to ensure manifest can locate it
        abs_mat_path = os.path.abspath(mat_path)
        manifest_file.write(f"{abs_mat_path}\t{segment_samples}\t{label_str}\n")